검증 지역과 행정 지역 매칭하여 
검증 지역 리스트에 읍면동 코드를 추가 후 저장

In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
# ASOS 관측 데이터
weather = pd.read_csv('../data/기상/OBS_ASOS_DD_20210601231517.csv', encoding = 'cp949')
# ASOS 관측소 메타 데이터
coor = pd.read_csv('../data/기상/META_관측지점정보_20210603224446.csv',encoding='cp949')
# 산사태 발생 이력 데이터
land = pd.read_csv('../data/산사태 발생이력.csv', encoding = 'cp949')
# 행정동 경계 데이터
boundary = gpd.read_file('../data/행정동 경계/umd.shp',encoding='utf-8')
# 결과 데이터
test = pd.read_csv('../data/참가번호.csv',encoding='cp949')

/home/suna/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (60) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
BB = pd.read_excel('../data/(공공협력분야) 데이터 정의서.xlsx',sheet_name='경상도 지역정보')
BB.columns = BB.iloc[0]
BB = BB.iloc[1:]

In [4]:
landS_list = test.drop_duplicates(['sd','sgg','umd'])[['sd','sgg','umd']]

In [5]:
print('Total',len(landS_list))
print('중복제거',len(set(landS_list['umd'])))

Total 299
중복제거 293


In [6]:
land_total = list(landS_list['umd'])
land_only = [i for i in land_total if land_total.count(i)==1]
land_more1 = [i for i in land_total if land_total.count(i)!=1]

In [7]:
bound_total = list(boundary['ADM_DR_NM'])
bound_only = [i for i in bound_total if bound_total.count(i)==1]
bound_more1 = [i for i in bound_total if bound_total.count(i)!=1]

In [8]:
bound_list = pd.merge(boundary,BB[['읍면동코드','시도명칭','시군구명칭','읍면동명칭',]],left_on='ADM_DR_CD',right_on='읍면동코드',how='left')
bound_list = bound_list.drop(['BASE_DATE','geometry'],axis=1)

검증 지역이 행정 지역에 해당되면 동코드를 추가하고  
해당되지 않으면 print하여 살펴본다  

In [9]:
landS_list['ADM_DR_CD'] = 0
for i in range(len(landS_list)):
    umd = landS_list['umd'].iloc[i].strip(' ')
    sgg = landS_list['sgg'].iloc[i].strip(' ')
    sd = landS_list['sd'].iloc[i].strip(' ')
    tmp = bound_list[bound_list['읍면동명칭']==umd]
    tmp = tmp[tmp['시군구명칭']==sgg]
    tmp = tmp[tmp['시도명칭']==sd]
    if len(tmp)==1:
        landS_list['ADM_DR_CD'].iloc[i] = tmp['ADM_DR_CD'].iloc[0]
    else:
        print(sd,sgg,umd)

/home/suna/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


경상남도 거제시 삼거동
경상남도 양산시 어곡동
경상남도 양산시 주진동
경상북도 경주시 시동
경상북도 김천시 덕곡동
경상북도 영주시 조와동


동코드가 겹치는 검증 지역 존재  
이들은 결과가 서로 똑같을 것이다  

In [10]:
tmp = list(landS_list['ADM_DR_CD'])
for i in landS_list['ADM_DR_CD']:
    if tmp.count(i)>1:
        print(i,tmp.count(i))

0 6
3835035 2
3840032 2
3840032 2
3734034 2
3734034 2
3701234 2
3701232 2
3835035 2
0 6
0 6
0 6
0 6
0 6
3701234 2
3701232 2


test 데이터에는 삼거동이 있는데 행정동 데이터에는 없다  
찾아보니 삼거동은 거제의 법정동이라고 한다  
삼거동은 거제면으로 변경하여 사용하기로 한다  
어곡동은 강서동에 해당하는 법정동이다  
어곡동은 강서동으로 변경하여 사용하기로 한다  
주진동은 소주동에 해당하므로 소주동으로 변경한다  
시동 > 불국동  
덕곡동 > 지좌동  
조와동 > 상망동

In [11]:
landS_list = landS_list.replace('삼거동','거제면')
landS_list = landS_list.replace('어곡동','강서동')
landS_list = landS_list.replace('주진동','소주동')
landS_list = landS_list.replace('시동','불국동')
landS_list = landS_list.replace('덕곡동','지좌동')
landS_list = landS_list.replace('조와동','상망동')

In [12]:
landS_list

,sd,sgg,umd,ADM_DR_CD
0,경상남도,사천시,곤명면,3806036
3,경상남도,사천시,곤양면,3806035
6,경상남도,산청군,단성면,3837037
9,경상남도,산청군,오부면,3837032
12,경상남도,하동군,악양면,3836032
...,...,...,...,...
2154,경상북도,칠곡군,가산면,3739033
2158,경상북도,칠곡군,동명면,3739032
2162,경상북도,칠곡군,석적읍,3739013
2182,경상북도,포항시 북구,기계면,3701234


In [13]:
landS_list['ADM_DR_CD'] = 0
for i in range(len(landS_list)):
    umd = landS_list['umd'].iloc[i].strip(' ')
    sgg = landS_list['sgg'].iloc[i].strip(' ')
    sd = landS_list['sd'].iloc[i].strip(' ')
    tmp = bound_list[bound_list['읍면동명칭']==umd]
    tmp = tmp[tmp['시군구명칭']==sgg]
    tmp = tmp[tmp['시도명칭']==sd]
    if len(tmp)==1:
        landS_list['ADM_DR_CD'].iloc[i] = tmp['ADM_DR_CD'].iloc[0]
    else:
        print(sd,sgg,umd)

/home/suna/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [17]:
landS_list.to_csv('landS_with_CD.csv')